In [1]:
import numpy as np
import pandas as pd
import re
import gensim
from gensim import corpora, models, similarities

In [4]:
emails = pd.read_csv('Emails.csv')
emails = emails[['Id', 'ExtractedBodyText']].dropna()

In [10]:
# clean the email texts
def clean_text(text):
    """ Clean the text of each email """
    text = text.replace('\n'," ") #remove line break
    text = re.sub(r"-", " ", text) #replace hypens with space
    text = re.sub(r"\d+/\d+/\d+", "", text) #remove date
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text) #remove times
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #remove email addresses
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) #removes web addresses
    clndoc = ''
    for letter in text:
        if letter.isalpha() or letter==' ':
            clndoc+=letter
    text = ' '.join(word for word in clndoc.split() if len(word)>1)
    return text    

In [11]:
email_body = emails.ExtractedBodyText.apply(clean_text)

In [12]:
email_body.head()

1    Thursday March PM Latest How Syria is aiding Q...
2                                                  Thx
4    Friday March PM Huma Abedin Fw Latest How Syri...
5    Pis print Wednesday September PM Fw Meet The R...
7    Friday March PM Huma Abedin Fw Latest How Syri...
Name: ExtractedBodyText, dtype: object

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
stopwords = tfidf.get_stop_words()

In [9]:
email_text_standardize = lambda email: [word for word in email.lower().split() if word not in stopwords]
email_texts = map(email_text_standardize, email_body.values)
                  

In [16]:
dictionary = corpora.Dictionary(email_texts)
corpus = [dictionary.doc2bow(email) for email in email_texts]

In [36]:
def build_topic_model(num_topics):
    email_lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=5)
    print email_lda.show_topics()
    return email_lda

In [37]:
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(build_topic_model(5), corpus, dictionary)

[(0, u'0.005*"state" + 0.005*"people" + 0.004*"new" + 0.004*"government" + 0.004*"security" + 0.004*"american" + 0.004*"states" + 0.003*"united" + 0.003*"said" + 0.003*"work"'), (1, u'0.033*"fyi" + 0.025*"pm" + 0.017*"pls" + 0.016*"ok" + 0.012*"print" + 0.012*"fw" + 0.012*"cheryl" + 0.009*"huma" + 0.009*"mills" + 0.009*"thx"'), (2, u'0.070*"pm" + 0.037*"office" + 0.028*"secretarys" + 0.021*"meeting" + 0.019*"room" + 0.014*"state" + 0.013*"department" + 0.013*"arrive" + 0.013*"route" + 0.012*"en"'), (3, u'0.009*"just" + 0.009*"know" + 0.008*"state" + 0.008*"im" + 0.007*"tomorrow" + 0.006*"today" + 0.006*"good" + 0.006*"like" + 0.006*"talk" + 0.006*"want"'), (4, u'0.007*"obama" + 0.006*"said" + 0.006*"mr" + 0.006*"party" + 0.005*"president" + 0.005*"new" + 0.004*"white" + 0.004*"house" + 0.003*"republican" + 0.003*"senate"')]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      41.730270        1       1  0.166900  0.057841
4      31.055178        1       2  0.141091  0.038953
3      16.007119        1       3  0.033080 -0.119315
2       6.415973        1       4 -0.190670  0.194077
1       4.791460        1       5 -0.150400 -0.171556, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
13535  Default  1838.000000          pm  1838.000000  30.0000  30.0000
22936  Default   884.000000      office   884.000000  29.0000  29.0000
27641  Default   542.000000  secretarys   542.000000  28.0000  28.0000
26306  Default   509.000000         fyi   509.000000  27.0000  27.0000
25778  Default   630.000000     meeting   630.000000  26.0000  26.0000
24372  Default  1324.000000       state  1324.000000  25.0000  25.0000
3433   Default   422.000000        room   422.000000  24.0000  24.0000
2932   Default   614.000000  department   614.000000  23.0000  23.0000
23188  Default   293.000000         pls   293.000000  22.0000  22.0000
21513  Default   930.000000        time   930.000000  21.0000  21.0000
13174  Default   249.000000          ok   249.000000  20.0000  20.0000
5424   Default   343.000000  conference   343.000000  19.0000  19.0000
11934  Default   310.000000     private   310.000000  18.0000  18.0000
25110  Default   248.000000      arrive   248.000000  17.0000  17.0000
10241  Default   248.000000       route   248.000000  16.0000  16.0000
9274   Default   239.000000          en   239.000000  15.0000  15.0000
921    Default   235.000000      depart   235.000000  14.0000  14.0000
28039  Default   686.000000        know   686.000000  13.0000  13.0000
10819  Default   436.000000          im   436.000000  12.0000  12.0000
18628  Default   217.000000       print   217.000000  11.0000  11.0000
12410  Default   609.000000          mr   609.000000  10.0000  10.0000
8883   Default   847.000000        just   847.000000   9.0000   9.0000
17051  Default   208.000000      cheryl   208.000000   8.0000   8.0000
2327   Default   364.000000    tomorrow   364.000000   7.0000   7.0000
9691   Default   178.000000          fw   178.000000   6.0000   6.0000
8994   Default   221.000000         thx   221.000000   5.0000   5.0000
21803  Default   196.000000   residence   196.000000   4.0000   4.0000
11063  Default   219.000000      sunday   219.000000   3.0000   3.0000
4539   Default   510.000000     foreign   510.000000   2.0000   2.0000
20085  Default   344.000000    minister   344.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
23624   Topic5    15.479797   forwarded    16.737856   2.9602  -6.8458
27710   Topic5     9.280502      scroll    10.052663   2.9584  -7.3574
26685   Topic5   126.360890    sullivan   137.733311   2.9522  -4.7462
22719   Topic5    18.896400      mchale    20.624911   2.9508  -6.6464
17469   Topic5   116.782771         pis   135.769166   2.8877  -4.8250
23188   Topic5   244.391288         pls   293.768007   2.8543  -4.0866
18628   Topic5   180.234557       print   217.945268   2.8484  -4.3911
17051   Topic5   171.093008      cheryl   208.627244   2.8400  -4.4431
9121    Topic5   138.176627        huma   183.285203   2.7558  -4.6568
14090   Topic5    46.045794         wjc    54.996995   2.8607  -5.7557
19252   Topic5    77.707609         add   112.387190   2.6693  -5.2324
5292    Topic5   105.736491    saturday   164.295208   2.5976  -4.9244
8994    Topic5   133.172189         thx   221.145956   2.5312  -4.6937
2496    Topic5    55.748034  blackberry    77.354256   2.7108  -5.5645
7134    Topic5    50.515940        copy    69.201594   2.7236  -5.6630
11063   Topic5   126.844617      sunday   219.230397   2.4912  -4.7424
15      Topic5    90.642943   wednesday   149.551133   2.5376  -5.0784
21321   Topic5    93.508745  

In [38]:
pyLDAvis.gensim.prepare(build_topic_model(10), corpus, dictionary)

[(0, u'0.015*"mr" + 0.011*"said" + 0.009*"obama" + 0.007*"president" + 0.007*"new" + 0.007*"party" + 0.005*"white" + 0.005*"david" + 0.005*"clinton" + 0.005*"secretary"'), (1, u'0.034*"pls" + 0.023*"print" + 0.016*"doc" + 0.014*"state" + 0.013*"pis" + 0.010*"huma" + 0.010*"abedin" + 0.010*"date" + 0.008*"press" + 0.008*"add"'), (2, u'0.006*"republican" + 0.006*"senate" + 0.005*"obama" + 0.005*"care" + 0.005*"democrats" + 0.005*"republicans" + 0.005*"health" + 0.004*"right" + 0.004*"party" + 0.004*"vote"'), (3, u'0.083*"pm" + 0.047*"office" + 0.034*"secretarys" + 0.026*"meeting" + 0.024*"room" + 0.020*"state" + 0.017*"department" + 0.015*"arrive" + 0.015*"route" + 0.015*"en"'), (4, u'0.017*"israel" + 0.014*"israeli" + 0.008*"jewish" + 0.007*"netanyahu" + 0.007*"palestinian" + 0.006*"peace" + 0.006*"deal" + 0.005*"said" + 0.005*"minister" + 0.005*"palestinians"'), (5, u'0.037*"pm" + 0.015*"cheryl" + 0.014*"fw" + 0.013*"sunday" + 0.012*"mills" + 0.012*"sullivan" + 0.011*"monday" + 0.011*"

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      30.487498        1       1  0.159427 -0.089189
2      16.854388        1       2  0.132930 -0.105827
0      10.781526        1       3  0.126758 -0.056455
8       9.844509        1       4  0.097470  0.181233
9       9.048794        1       5  0.061230  0.192072
4       7.025269        1       6  0.064386 -0.085510
3       5.772736        1       7 -0.228644 -0.125306
5       4.304568        1       8 -0.153034  0.159520
1       3.551546        1       9 -0.193713 -0.020270
7       2.329166        1      10 -0.066809 -0.050267, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
13535  Default  2022.000000           pm  2022.000000  30.0000  30.0000
22936  Default   954.000000       office   954.000000  29.0000  29.0000
27641  Default   594.000000   secretarys   594.000000  28.0000  28.0000
24372  Default  1345.000000        state  1345.000000  27.0000  27.0000
23188  Default   386.000000          pls   386.000000  26.0000  26.0000
25778  Default   660.000000      meeting   660.000000  25.0000  25.0000
3433   Default   455.000000         room   455.000000  24.0000  24.0000
26306  Default   650.000000          fyi   650.000000  23.0000  23.0000
2932   Default   639.000000   department   639.000000  22.0000  22.0000
12410  Default   514.000000           mr   514.000000  21.0000  21.0000
478    Default   423.000000       israel   423.000000  20.0000  20.0000
18628  Default   281.000000        print   281.000000  19.0000  19.0000
16542  Default   324.000000      israeli   324.000000  18.0000  18.0000
25110  Default   272.000000       arrive   272.000000  17.0000  17.0000
10241  Default   271.000000        route   271.000000  16.0000  16.0000
13174  Default   320.000000           ok   320.000000  15.0000  15.0000
9274   Default   262.000000           en   262.000000  14.0000  14.0000
921    Default   258.000000       depart   258.000000  13.0000  13.0000
11934  Default   331.000000      private   331.000000  12.0000  12.0000
5424   Default   360.000000   conference   360.000000  11.0000  11.0000
28039  Default   641.000000         know   641.000000  10.0000  10.0000
3701   Default   222.000000          yes   222.000000   9.0000   9.0000
17051  Default   240.000000       cheryl   240.000000   8.0000   8.0000
8883   Default   803.000000         just   803.000000   7.0000   7.0000
21803  Default   215.000000    residence   215.000000   6.0000   6.0000
21513  Default   924.000000         time   924.000000   5.0000   5.0000
10819  Default   406.000000           im   406.000000   4.0000   4.0000
22897  Default   649.000000         work   649.000000   3.0000   3.0000
11063  Default   236.000000       sunday   236.000000   2.0000   2.0000
9691   Default   215.000000           fw   215.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
11120  Topic10     7.842263         agis     8.705237   3.6553  -6.8045
22325  Topic10     7.660299       smiths     8.528182   3.6523  -6.8279
25789  Topic10     7.547048        solis     8.410293   3.6514  -6.8428
3965   Topic10     7.771629         auca     8.672025   3.6500  -6.8135
903    Topic10    25.621129       corker    28.848922   3.6410  -5.6206
3701   Topic10   124.435620          yes   222.378280   3.1791  -4.0402
3210   Topic10    11.426501      thanked    13.822565   3.5693  -6.4281
5670   Topic10    20.465693        abbas    28.840669   3.4166  -5.8452
6325   Topic10    18.829501       flight    26.386931   3.4222  -5.9286
13721  Topic10    19.110235          usg    27.992040   3.3780  -5.9138
2559   Topic10    10.984132   thoughtful    13.546292   3.5500  -6.4675
19305  Topic10    12.920840        levin    18.210360   3.4165  -6.3052
16035  Topic10    20.584419      assange    49.808426   2.8760  -5.83

In [39]:
pyLDAvis.gensim.prepare(build_topic_model(15), corpus, dictionary)

[(3, u'0.009*"obama" + 0.007*"american" + 0.007*"president" + 0.006*"new" + 0.006*"said" + 0.005*"war" + 0.005*"afghanistan" + 0.005*"military" + 0.005*"political" + 0.004*"administration"'), (0, u'0.024*"print" + 0.023*"pls" + 0.014*"pis" + 0.011*"qddr" + 0.007*"speech" + 0.007*"sbwhoeop" + 0.006*"new" + 0.006*"read" + 0.006*"thx" + 0.005*"add"'), (2, u'0.012*"state" + 0.009*"united" + 0.009*"states" + 0.008*"security" + 0.006*"people" + 0.006*"government" + 0.005*"diplomacy" + 0.005*"department" + 0.005*"secretary" + 0.005*"nuclear"'), (1, u'0.012*"know" + 0.007*"let" + 0.006*"late" + 0.005*"food" + 0.005*"good" + 0.005*"statement" + 0.005*"going" + 0.004*"iii" + 0.004*"plane" + 0.004*"thx"'), (10, u'0.019*"percent" + 0.015*"israeli" + 0.015*"israel" + 0.011*"party" + 0.010*"labour" + 0.008*"poll" + 0.008*"jewish" + 0.008*"palestinian" + 0.007*"obama" + 0.007*"voters"'), (7, u'0.093*"pm" + 0.049*"office" + 0.037*"secretarys" + 0.027*"meeting" + 0.026*"room" + 0.024*"state" + 0.018*"d

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      20.469910        1       1  0.239621  0.007811
4      20.152114        1       2  0.165222  0.093685
2      15.359455        1       3  0.204632  0.114762
9       6.843951        1       4  0.074442 -0.181162
8       5.661654        1       5 -0.111344  0.206868
7       5.520216        1       6 -0.061318  0.086568
5       4.675869        1       7 -0.162689  0.168446
10      4.545530        1       8  0.063781 -0.123828
13      4.006882        1       9  0.036863 -0.095860
6       3.757361        1      10  0.065653  0.026042
0       2.174872        1      11 -0.067439  0.028910
11      2.150265        1      12 -0.137478 -0.084482
1       1.922607        1      13 -0.064327 -0.004190
12      1.458222        1      14 -0.090075 -0.148170
14      1.301093        1      15 -0.155542 -0.095401, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
13535  Default  2098.000000          pm  2098.000000  30.0000  30.0000
22936  Default   975.000000      office   975.000000  29.0000  29.0000
24372  Default  1400.000000       state  1400.000000  28.0000  28.0000
26306  Default   632.000000         fyi   632.000000  27.0000  27.0000
27641  Default   613.000000  secretarys   613.000000  26.0000  26.0000
25778  Default   688.000000     meeting   688.000000  25.0000  25.0000
3433   Default   470.000000        room   470.000000  24.0000  24.0000
2932   Default   663.000000  department   663.000000  23.0000  23.0000
13174  Default   327.000000          ok   327.000000  22.0000  22.0000
15325  Default   343.000000      senate   343.000000  21.0000  21.0000
478    Default   416.000000      israel   416.000000  20.0000  20.0000
28039  Default   670.000000        know   670.000000  19.0000  19.0000
25626  Default   690.000000       house   690.000000  18.0000  18.0000
6804   Default   337.000000       press   337.000000  17.0000  17.0000
23188  Default   331.000000         pls   331.000000  16.0000  16.0000
2327   Default   358.000000    tomorrow   358.000000  15.0000  15.0000
25110  Default   280.000000      arrive   280.000000  14.0000  14.0000
10241  Default   280.000000       route   280.000000  13.0000  13.0000
7978   Default   852.000000       obama   852.000000  12.0000  12.0000
28024  Default   281.000000     percent   281.000000  11.0000  11.0000
9274   Default   271.000000          en   271.000000  10.0000  10.0000
921    Default   267.000000      depart   267.000000   9.0000   9.0000
3242   Default   526.000000       party   526.000000   8.0000   8.0000
16542  Default   315.000000     israeli   315.000000   7.0000   7.0000
11934  Default   339.000000     private   339.000000   6.0000   6.0000
18628  Default   229.000000       print   229.000000   5.0000   5.0000
22897  Default   640.000000        work   640.000000   4.0000   4.0000
25617  Default   540.000000        good   540.000000   3.0000   3.0000
3701   Default   193.000000         yes   193.000000   2.0000   2.0000
5884   Default   277.000000        case   277.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
20061  Topic15     7.660078       berry     8.531133   4.2343  -6.2457
12900  Topic15     7.550572  struggling     8.416845   4.2334  -6.2601
8971   Topic15     7.011995      hebrew     7.879005   4.2254  -6.3341
10099  Topic15     6.410776     ebeling     7.281846   4.2146  -6.4237
6077   Topic15     6.367825         suu     7.234611   4.2143  -6.4304
1917   Topic15     6.367825        aung     7.234611   4.2143  -6.4304
5353   Topic15     6.352945    thorough     7.219540   4.2141  -6.4328
15600  Topic15     6.060275    moriarty     6.926657   4.2083  -6.4799
728    Topic15     5.863957    tracking     6.730293   4.2042  -6.5129
25833  Topic15    20.431752        mins    23.495543   4.

In [40]:
pyLDAvis.gensim.prepare(build_topic_model(20), corpus, dictionary)

[(6, u'0.039*"pm" + 0.038*"pls" + 0.027*"print" + 0.026*"cheryl" + 0.024*"fw" + 0.019*"mills" + 0.018*"pis" + 0.015*"sullivan" + 0.013*"fyi" + 0.013*"thx"'), (18, u'0.050*"release" + 0.020*"kurdistan" + 0.018*"melanne" + 0.015*"verveer" + 0.013*"sent" + 0.012*"blackberry" + 0.010*"sudan" + 0.010*"astoria" + 0.010*"arturo" + 0.009*"mail"'), (12, u'0.014*"mcchrystal" + 0.010*"taliban" + 0.010*"afghanistan" + 0.008*"said" + 0.008*"bloomberg" + 0.007*"al" + 0.006*"troops" + 0.006*"mayor" + 0.006*"holbrooke" + 0.005*"military"'), (4, u'0.018*"message" + 0.016*"secretary" + 0.014*"memo" + 0.010*"statement" + 0.010*"hillary" + 0.009*"copy" + 0.009*"sheet" + 0.009*"letter" + 0.009*"lauren" + 0.008*"follow"'), (8, u'0.017*"diplomacy" + 0.010*"state" + 0.009*"development" + 0.009*"treaty" + 0.007*"department" + 0.006*"states" + 0.005*"marie" + 0.005*"policy" + 0.005*"postconflict" + 0.005*"officials"'), (17, u'0.009*"people" + 0.008*"security" + 0.007*"united" + 0.007*"states" + 0.006*"work" + 0

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
17     15.737926        1       1  0.232624 -0.024500
1      13.604918        1       2  0.200167 -0.061530
16     12.567238        1       3  0.194659 -0.041538
19      7.363443        1       4  0.179003 -0.041254
2       6.327356        1       5  0.126021 -0.039357
0       6.283414        1       6  0.095860  0.215436
15      5.719000        1       7 -0.057254 -0.000599
3       5.421988        1       8  0.037311 -0.000067
10      4.272351        1       9  0.048048 -0.050422
9       3.963018        1      10 -0.022344  0.290445
12      3.584234        1      11  0.057530 -0.034495
8       3.488431        1      12  0.105825 -0.058565
6       2.845008        1      13 -0.106855  0.210637
14      2.700237        1      14 -0.133266 -0.073503
4       1.453502        1      15 -0.156775  0.090806
13      1.233543        1      16 -0.148505 -0.097190
11      1.074263        1      17 -0.172584 -0.107868
7       0.881679        1      18 -0.151994 -0.047909
5       0.774909        1      19 -0.147441 -0.051088
18      0.703543        1      20 -0.180029 -0.077438, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
13535  Default  2145.000000           pm  2145.000000  30.0000  30.0000
24372  Default  1541.000000        state  1541.000000  29.0000  29.0000
22936  Default   997.000000       office   997.000000  28.0000  28.0000
27641  Default   642.000000   secretarys   642.000000  27.0000  27.0000
26306  Default   700.000000          fyi   700.000000  26.0000  26.0000
3433   Default   490.000000         room   490.000000  25.0000  25.0000
2932   Default   708.000000   department   708.000000  24.0000  24.0000
25778  Default   695.000000      meeting   695.000000  23.0000  23.0000
478    Default   389.000000       israel   389.000000  22.0000  22.0000
23188  Default   335.000000          pls   335.000000  21.0000  21.0000
25626  Default   739.000000        house   739.000000  20.0000  20.0000
1953   Default   610.000000    secretary   610.000000  19.0000  19.0000
13174  Default   308.000000           ok   308.000000  18.0000  18.0000
28039  Default   665.000000         know   665.000000  17.0000  17.0000
16542  Default   296.000000      israeli   296.000000  16.0000  16.0000
17802  Default   256.000000          doc   256.000000  15.0000  15.0000
10819  Default   420.000000           im   420.000000  14.0000  14.0000
2327   Default   344.000000     tomorrow   344.000000  13.0000  13.0000
25110  Default   293.000000       arrive   293.000000  12.0000  12.0000
18628  Default   248.000000        print   248.000000  11.0000  11.0000
10241  Default   293.000000        route   293.000000  10.0000  10.0000
5884   Default   330.000000         case   330.000000   9.0000   9.0000
12410  Default   567.000000           mr   567.000000   8.0000   8.0000
9274   Default   283.000000           en   283.000000   7.0000   7.0000
17051  Default   239.000000       cheryl   239.000000   6.0000   6.0000
921    Default   279.000000       depart   279.000000   5.0000   5.0000
7978   Default   855.000000        obama   855.000000   4.0000   4.0000
15325  Default   360.000000       senate   360.000000   3.0000   3.0000
7459   Default   226.000000         date   226.000000   2.0000   2.0000
25617  Default   532.000000         good   532.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
18170  Topic20     8.482464     gonzalez     9.355961   4.8588  -5.5289
23483  Topic20     7.278360  westerwelle     8.152521   4.8434  -5.6820
28064  Topic20     6.710952      murdoch     7.584402   4.8344  -5.7631
19692  Topic20     6.461772         bbdb     7.335209   4.8300  -5.8010
6077   Topic20     6.384178          suu     7.259430   4.8283  -5.8130
1917   Topic20     6.384178      